In [123]:
import jinja2
import os
import cv2
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup, Tag

In [124]:
desc=pd.read_excel('images_desc.xlsx',encoding='utf-8',header=0)
desc['index']=desc['index'].astype(str).str.strip()

In [125]:
def describe_images(folder):
    desc=[]
    for i in os.listdir(folder):
        image =folder+'/'+i
        #print(image)
        img =  cv2.imread(image)
        height, width, depth = img.shape
        desc.append({'file':image,'w':width,'h':height,'aspect_ratio':height/width})
        
    desc=pd.DataFrame(desc)
    
    return desc

In [126]:
thumbs=describe_images('images/tmp/thumbs')
thumbs.columns=['tfile','tw','th','taspect_ratio']
thumbs['index']=thumbs['tfile'].str.split('/',-1,expand=True)[3].str.split('-',expand=True)[0].str.strip('.JPG').str.strip()

In [127]:
fulls=describe_images('images/tmp/fulls')
fulls['index']=fulls['file'].str.split('/',-1,expand=True)[3].str.split('-',expand=True)[0].str.strip('.JPG').str.strip()

In [128]:
tags=desc.merge(thumbs,on='index',how='left').merge(fulls,on='index',how='left')
tags=tags.rename(columns={'Description':'caption','Title':'Title','tfile':'thumb','file':'full'})
tags.loc[tags['index']=='intro','index']=str(0)

In [129]:
tags=tags.sort_values(by='index').to_dict(orient='records')

In [130]:
tags

[{'index': '0',
  'Title': 'Project Description',
  'caption': 'Examples of work, during my tenure as Co-Director of the project: "Developing socio-spatial clustering algorithms to unveil urban dynamics from near real-time non-conventional data sources for urban planning, management, and monitoring of cities". Tasks included the development of the proposal and supervision of the project for a year. Credits for collaborators: Horacio Samaniego, Boris Sotomayor, Mauricio Cisternas (UACH), Pablo Valenzuela, Fernando del Campo, Sebastián Pereira, Amanda Ibsen (INRIA).',
  'thumb': 'images/tmp/thumbs/intro.JPG',
  'tw': 360.0,
  'th': 224.0,
  'taspect_ratio': 0.6222222222222222,
  'full': 'images/tmp/fulls/intro.JPG',
  'w': 1200.0,
  'h': 945.0,
  'aspect_ratio': 0.7875},
 {'index': '1',
  'Title': 'Proposal and Methods',
  'caption': 'The initial proposal consisted in the exploration of several possibilities of Urban Boundaries as defined by socio-spatial clustering algorithms (based on 

In [131]:
from jinja2 import Template
with open('template.html') as file_:
    template = Template(file_.read())

In [132]:
rendered=template.render(tags=tags)

In [ ]:
with open('index.html', 'w') as f:
    f.write(rendered)